## Reconstruct new models without embedding

In [6]:
from megnet.models import MEGNetModel
from megnet.data.graph import GaussianDistance
from megnet.data.crystal import CrystalGraph
import numpy as np

# Pretrained model
model_pretrained = MEGNetModel.from_file('/home/deep/megnet-master/mvl_models/mp-2018.6.1/band_gap_regression.hdf5')

# New model with length 16 atom feature
cg = CrystalGraph(bond_converter=GaussianDistance(np.linspace(0, 5, 100), 0.5), cutoff=4)
model_new = MEGNetModel(100, 2, 16, graph_converter=cg)

2024-01-22 18:44:18.751608: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-22 18:44:18.764809: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-22 18:44:18.764906: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-22 18:44:18.765414: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [20]:
# Get pretrained weights
weights = model_pretrained.get_weights()
# The first weight is the element embedding
emb = weights[0] 
# The new model has exact the same layers except the embedding layer
model_new.set_weights(weights[1:]) 

In [36]:
len(weights)

105

In [27]:
weights[0].shape

(95, 16)

In [28]:
# from pymatgen import MPRester
# mpr = MPRester()

from pymatgen.ext.matproj import MPRestError, TaskType, _MPResterLegacy
# rester = _MPResterLegacy(api_key='iAihdZzrZYLQKZms1S43De90NiNK6ABB') # mp-api style
mpr = _MPResterLegacy('iTPrDnB1NuSywGXI')  # legacy style get the mp data

s = mpr.get_structure_by_material_id('mp-129')

/root/anaconda3/envs/megnet/lib/python3.8/site-packages/pymatgen/ext/matproj.py:183: UserWarning: You are using the legacy MPRester. This version of the MPRester will no longer be updated. To access the latest data with the new MPRester, obtain a new API key from https://materialsproject.org/api and consult the docs at https://docs.materialsproject.org/ for more information.
  warnings.warn(


In [29]:
s

Structure Summary
Lattice
    abc : 2.743237657484856 2.743237657484856 2.743237657484856
 angles : 109.47122063449069 109.47122063449069 109.47122063449069
 volume : 15.891628753050979
      A : -1.583809 1.583809 1.583809
      B : 1.583809 -1.583809 1.583809
      C : 1.583809 1.583809 -1.583809
    pbc : True True True
PeriodicSite: Mo (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]

In [33]:
len(s)

1

In [30]:
# Pretrained model prediction
model_pretrained.predict_structure(s)

2024-01-22 18:51:04.535940: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


array([0.03076014], dtype=float32)

In [31]:
# Concert manually the atomic number into an atomic feature
graph = model_new.graph_converter.convert(s)
graph['atom'] = [emb[i] for i in graph['atom']]

In [32]:
graph

{'atom': [array([ 0.22034007,  0.37071624,  0.0516335 , -0.18536569, -0.0900702 ,
          0.08451756,  0.3357563 ,  0.9284428 , -0.09759796,  0.7980744 ,
          0.13921405,  0.34473124,  0.1980346 ,  0.01140447, -0.23631614,
         -0.28500003], dtype=float32)],
 'bond': array([2.7432377, 2.7432377, 2.7432377, 2.7432377, 3.167618 , 2.7432377,
        2.7432377, 3.167618 , 2.7432377, 3.167618 , 2.7432377, 3.167618 ,
        3.167618 , 3.167618 ], dtype=float32),
 'state': array([[0., 0.]], dtype=float32),
 'index1': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32),
 'index2': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)}

In [34]:
# Same prediction results
model_new.predict_graph(graph)

array([0.03076014], dtype=float32)